In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/hotpotscienceqa-entailment/sciencehotpotqa_single_stage_generated.csv


In [2]:
df = pd.read_csv('/kaggle/input/hotpotscienceqa-entailment/sciencehotpotqa_single_stage_generated.csv')
df.head()

,question,correct_answers,incorrect_answers,original_context,all_sentences,document
0,Which of the following statements accurately d...,['Cadmium Chloride is slightly soluble in wate...,['Cadmium Chloride is highly soluble in alcoho...,Cadmium Chloride is slightly soluble in this c...,['Cadmium chloride is a white crystalline comp...,Cadmium Chloride is slightly soluble in this c...
1,What characteristics do Kniphofia and Baptisia...,['Both Kniphofia and Baptisia produce flowerin...,['Kniphofia and Baptisia are both classified a...,What type of vegetation does Kniphofia and Bap...,['The Dinaric calcareous silver fir forests ar...,What type of vegetation does Kniphofia and Bap...
2,Which of the following statements best describ...,['Boreo-arctic Montane species are typically f...,['Boreo-arctic Montane species are primarily f...,What type of species is a Boreo-arctic Montane...,"[""The five main latitude regions of the Earth'...",What type of species is a Boreo-arctic Montane...
3,What are the shared characteristics of Fotherg...,['Fothergilla and Clerodendrum both belong to ...,['Fothergilla and Clerodendrum are both native...,What type of vegetation does Fothergilla and C...,"['Clerodendrum infortunatum, the hill glory bo...",What type of vegetation does Fothergilla and C...
4,Which statements accurately reflect the charac...,['Dendranthema boreale belongs to the Asterace...,['Dendranthema boreale is classified in the Or...,What are the plants in the same genus as the D...,['Navarretia is a genus of about 30 species of...,What are the plants in the same genus as the D...


In [3]:
import os
import json
import time
import pandas as pd
import random
from openai import OpenAI, RateLimitError
from tenacity import retry, wait_random_exponential, stop_after_attempt
from datasets import load_dataset
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

# --- 1. Setup and API Call Functions ---

openai_key = user_secrets.get_secret("openai")
if not openai_key:
    raise ValueError("OPENAI_API_KEY environment variable not set.")
client = OpenAI(api_key=openai_key)


def call_gpt4o_mini(messages):
    """Sends a prompt to the GPT-4o-mini API with robustness for errors."""
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            response_format={"type": "json_object"},
            temperature=0.7,
            n=1
        )
        content = response.choices[0].message.content
        return json.loads(content)
    except RateLimitError as e:
        print(f"Rate limit error: {e}. Retrying...")
        raise
    except json.JSONDecodeError as e:
        print(f"Received invalid JSON from API: {e}. Retrying...")
        raise e
    except Exception as e:
        print(f"An unexpected error occurred during the API call: {e}")
        raise e

# --- 2. Data Loading and Filtering ---

def load_and_filter_scienceqa_data():
    """
    Loads the ScienceQA dataset, filters for a specific subset, and then
    further filters for text-only questions with a valid lecture/solution context.
    """
    print("Loading ScienceQA dataset...")
    df = pd.read_csv('/kaggle/input/hotpotscienceqa-entailment/sciencehotpotqa_single_stage_generated.csv')
    

    print(f"Loaded and filtered {len(df)} text-only questions.")
    return df

def construct_single_stage_prompt(context, question, correct_answers, incorrect_answers):
    """
    Creates multi-step reasoning questions where each option is a compound-compound statement:
    - The entire option is joined by AND/OR
    - Each side of the AND/OR is itself a fused multi-step fact
    """
    user_content = f"""
    You are creating a QA dataset. Your task: turn the provided single-step atomic facts into
    challenging, reasoning-based multiple-choice questions.

    Context: {context}
    Current Question: {question}
    Correct Atomic Facts: {correct_answers}
    Incorrect Atomic Facts: {incorrect_answers}

    **STRICT RULES**
    1. Each correct option MUST:
       - Create TWO distinct fused statements, where each fused statement combines at least TWO correct atomic facts into ONE coherent claim.
       - Join these two fused statements with AND or OR.
       - Both fused statements must themselves be multi-step (cannot be single-step claims).
       - The overall option must be true only if all relevant facts in both fused statements are true.
    2. Each incorrect option MUST:
       - Create TWO distinct fused statements, at least one of which contains at least one incorrect atomic fact.
       - Join them with AND or OR.
       - Both fused statements must be plausible in isolation but together make the whole statement false.
    3. No statement may be a simple listing of atomic facts.  
       - Each fused statement must integrate the facts into a single readable sentence, not just “Fact A AND Fact B”.
    4. Do not include phrases like “based on the given context” or anything implying the question relies on provided context.
       - The question should be fully answerable as a standalone fact-based question.

    **OUTPUT FORMAT (JSON only)**:
    ```json
    {{
      "question": "<standalone question text here>",
      "correct_combinations": ["...", "..."],
      "incorrect_combinations": ["...", "...", "..."]
    }}
    ```

    Now produce only the JSON.
    """
    messages = [
        {"role": "system", "content": "You are a QA system that strictly follows instructions."},
        {"role": "user", "content": user_content}
    ]
    return messages






# --- 4. Main Execution Block ---
def process_single_stage_pipeline(df, output_file_csv):
    """
    Orchestrates the single-stage pipeline and saves the final output to a CSV file.
    """
    print(f"Starting single-stage pipeline for {len(df)} rows.")
    final_output_list = []
    
    for i in range(len(df)):
        row = df.iloc[i]
        
        # This is where the model generates the question
        messages = construct_single_stage_prompt(row['original_context'],row['question'],row['correct_answers'],row['incorrect_answers'])
        try:
            result = call_gpt4o_mini(messages)
            
            # This is where we add the metadata to the final output
            result['correct_answers'] = row['correct_answers']
            result['incorrect_answers'] = row['incorrect_answers']
            result['original_context'] = row['original_context']
            result['all_sentences'] = row['all_sentences']
            result['document'] = row['document']

            final_output_list.append(result)
            print(f"  Processed row {i} successfully.")
        except Exception as e:
            print(f"  Failed to process row {i}. Skipping.")
        time.sleep(0.01)

    final_df = pd.DataFrame(final_output_list)
    final_df.to_csv(output_file_csv, index=False, encoding='utf-8')
    print(f"\nProcessing complete. {len(final_output_list)} final results saved to '{output_file_csv}'.")

if __name__ == '__main__':
    filtered_df = load_and_filter_scienceqa_data()
    if not filtered_df.empty:
        output_csv_file = 'sciencehotpot_logical_qa.csv'
        process_single_stage_pipeline(filtered_df, output_csv_file)

Loading ScienceQA dataset...
Loaded and filtered 1902 text-only questions.
Starting single-stage pipeline for 1902 rows.
  Processed row 0 successfully.
  Processed row 1 successfully.
  Processed row 2 successfully.
  Processed row 3 successfully.
  Processed row 4 successfully.
  Processed row 5 successfully.
  Processed row 6 successfully.
  Processed row 7 successfully.
  Processed row 8 successfully.
  Processed row 9 successfully.
  Processed row 10 successfully.
  Processed row 11 successfully.
  Processed row 12 successfully.
  Processed row 13 successfully.
  Processed row 14 successfully.
  Processed row 15 successfully.
  Processed row 16 successfully.
  Processed row 17 successfully.
  Processed row 18 successfully.
  Processed row 19 successfully.
  Processed row 20 successfully.
  Processed row 21 successfully.
  Processed row 22 successfully.
  Processed row 23 successfully.
  Processed row 24 successfully.
  Processed row 25 successfully.
  Processed row 26 successfully

In [4]:
ans_df = pd.read_csv('sciencehotpot_logical_qa.csv')

In [5]:
ans_df.head()

,question,correct_combinations,incorrect_combinations,correct_answers,incorrect_answers,original_context,all_sentences,document
0,Which of the following statements accurately d...,['Cadmium Chloride is slightly soluble in wate...,['Cadmium Chloride is highly soluble in alcoho...,['Cadmium Chloride is slightly soluble in wate...,['Cadmium Chloride is highly soluble in alcoho...,Cadmium Chloride is slightly soluble in this c...,['Cadmium chloride is a white crystalline comp...,Cadmium Chloride is slightly soluble in this c...
1,What characteristics do Kniphofia and Baptisia...,['Both Kniphofia and Baptisia produce flowerin...,['Kniphofia and Baptisia are classified as shr...,['Both Kniphofia and Baptisia produce flowerin...,['Kniphofia and Baptisia are both classified a...,What type of vegetation does Kniphofia and Bap...,['The Dinaric calcareous silver fir forests ar...,What type of vegetation does Kniphofia and Bap...
2,Which of the following statements best describ...,['Boreo-arctic Montane species are adapted to ...,['Boreo-arctic Montane species are primarily f...,['Boreo-arctic Montane species are typically f...,['Boreo-arctic Montane species are primarily f...,What type of species is a Boreo-arctic Montane...,"[""The five main latitude regions of the Earth'...",What type of species is a Boreo-arctic Montane...
3,What are the shared characteristics of Fotherg...,['Fothergilla is a genus of flowering plants t...,['Fothergilla and Clerodendrum are both native...,['Fothergilla and Clerodendrum both belong to ...,['Fothergilla and Clerodendrum are both native...,What type of vegetation does Fothergilla and C...,"['Clerodendrum infortunatum, the hill glory bo...",What type of vegetation does Fothergilla and C...
4,Which statements accurately reflect the classi...,['Dendranthema boreale is a flowering plant th...,['Dendranthema boreale is classified in the Or...,['Dendranthema boreale belongs to the Asterace...,['Dendranthema boreale is classified in the Or...,What are the plants in the same genus as the D...,['Navarretia is a genus of about 30 species of...,What are the plants in the same genus as the D...


In [6]:
for index, row in ans_df.iterrows():
    print("--- Row", index, "---")
    print("question:", row['question'])
    print("correct_combinations:", row['correct_combinations'])
    print("incorrect_combinations:", row['incorrect_combinations'])
    print("correct_answers:", row['correct_answers'])
    print("incorrect_answers:", row['incorrect_answers'])
    print()
    print("original_context:", row['original_context'])
    # print("all_sentences:", row['all_sentences'])
    # print("document:", row['document'])
    print("\n")
    print("\n")

--- Row 0 ---
question: Which of the following statements accurately describes the properties of Cadmium Chloride and Ethanol?
correct_combinations: ['Cadmium Chloride is slightly soluble in water AND Ethanol is highly soluble in water while being known as ethyl alcohol.', 'Ethanol has the chemical formula C2H5OH AND Cadmium Chloride is a hygroscopic solid that is slightly soluble in alcohol.']
incorrect_combinations: ['Cadmium Chloride is highly soluble in alcohol AND Ethanol is a compound with the chemical formula C2H6OH.', 'Ethanol is a solid that is not hygroscopic AND Cadmium Chloride is slightly soluble in alcohol.', 'Ethanol is known as ethyl alcohol AND Cadmium Chloride has a chemical formula different from C2H5OH.']
correct_answers: ['Cadmium Chloride is slightly soluble in water.', 'Ethanol is highly soluble in water.', 'Ethanol is also known as ethyl alcohol.', 'Ethanol has the chemical formula C2H5OH.']
incorrect_answers: ['Cadmium Chloride is highly soluble in alcohol.', '